# <center>Introduction to Curricular Simulations</center>

<center>
    <b>Gregory L. Heileman$^\dagger$, Jiacheng Zhang$^\ddagger$ and Hayden W. Free$^\bullet$</b> <br>
    $^\dagger$Department of Electrical & Computer Engineering <br>
    University of Arizona <br>
    heileman@arizona.edu <br>
    $^\ddagger$Department of Computer Science <br>
    jiachengzhang1@arizona.edu <br>
    University of Arizona<br>
    $^\bullet$Department of Computer Science <br>
    hayden.free@uky.edu <br>
    University of Kentucky
</center>

## 1. Introduction

This notebook demonstrates how to use the simulation capabilites that are included as a part of the [CurricularAnalytics toolbox](https://github.com/CurricularAnalytics/CurricularAnalytics.jl). If you would like to become more familiar with the notions behind curricular analytics, we suggest you read <cite data-cite="he:18">Heileman, et. al, (2018)</cite>, and also examine the Introduction to the Curricular Analytics Toolbox notebook that accompanies this notebook.

The simulation capabilites include the ability to simulate the flow of students through a curriculum, towards graduation, using discrete event simulation. Specifically, a population of students attempt to complete the selected curriculum, by taking courses in the order prescribed by the curriculum. At each step (semester) of the simulation a given student enrolls in a set of courses, earning either a passing of failing grade in each.  At the end of a given semester, if a student has passed all of the courses in the curriculum, they are deemed a graduate.  If a student has not yet gradauted, then they may stop out (according to a prescribed stop-out model), or enroll in the next set of courses available to them. One of the intended uses of these simulation capabilities is to estimate the impact that particular curricular changes or instructional improvements will have on student progress. 

The simulation framework (shown below) was orginally developed by <cite data-cite="hi:18">Hickman, (2014)</cite>, and  subsequent development has allowed it to be integrated into the CurricularAnalytics toolbox. 

<img src="SimulationFramework.png" width="600">

Notice that the simulation "engine" requires three inputs: a curriculum, a model for students, and a model for student peformance. The results of the simualtion are returned in an object that may be viewed using the `simulation_report()` function, as demonstrated below. 

In order to perform curricular simulations, first load the Curricular Analytics toolbox modules:

In [21]:
using CurricularAnalytics, CurricularVisualization

## 2. Setting up the Simulation Environment

The first thing we will do is read in a degree plan. The student in the simulation will attempt to complete the curriculum associated with the degree plan in the order that prescribed in the degree plan.  Specifically, in each semester each student will enroll in courses they have not yet taken in the order specified by the degree plan until they reach the maximum allowed number of credit hours.

### 2.1 Reading the Degree Plan

The following commands read in a degree plan stored in the CSV file format, and then display a visualization of the resulting plan.

In [2]:
AE_degree_plan = read_csv("Univ_of_Arizona-Aero.csv")
visualize(AE_degree_plan, notebook=true, scale=0.8)

WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src => "http://localhost:8156",:id => "curriculum",:events => Dict{Symbol,WebIO.JSString}(:load => WebIO.JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"Aerospace Engineering\",\"dp_name\":\"2019-20 Degree Plan\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"MATH 125\",\"metrics\":{\"centrality\":0,\"complexity\":38.0,\"blocking factor\":29,\"delay factor\":9.0},\"nameSub\":\"Calculus I w/ Applications\",\"id\":1,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CHEM 151\",\"metrics\":{\"centrality\":0,\"complexity\":5.0,\"blocking factor\":2,\"delay factor\":3.0},\"nameSub\":\"Gen Chemistry I\",\"id\":2,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"ENGL 101\",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"First-year Comp.\",\"id\":3,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1,\"target_id\":4,\"type\":\"CurriculumCorequisite\"}],\"name\":\"ENGR 102\",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"Intro to Engineering\",\"id\":4,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Tier I General Ed.\",\"id\":5,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":1,\"target_id\":6,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"MATH 129\",\"metrics\":{\"centrality\":387,\"complexity\":31.0,\"blocking factor\":22,\"delay factor\":9.0},\"nameSub\":\"Calculus II\",\"id\":6,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1,\"target_id\":7,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"AME 105\",\"metrics\":{\"centrality\":28,\"complexity\":16.0,\"blocking factor\":9,\"delay factor\":7.0},\"nameSub\":\"Intro. to MATLAB I\",\"id\":7,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1,\"target_id\":8,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"PHYS 141\",\"metrics\":{\"centrality\":277,\"complexity\":28.0,\"blocking factor\":19,\"delay factor\":9.0},\"nameSub\":\"Intro Mechanics\",\"id\":8,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":3,\"target_id\":9,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"ENGL 102\",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"First-year Comp.\",\"id\":9,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1,\"target_id\":10,\"type\":\"CurriculumCorequisite\"}],\"name\":\"ECE 175\",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"Programming for Eng. Apps.\",\"id\":10,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Tier I General Ed. 1\",\"id\":11,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":2},{\"name\":\"Term 3\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":8,\"target_id\":12,\"type\":\"CurriculumPrerequisite\"},{\"source_id\":6,\"target_id\":12,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"CE 214\",\"metrics\":{\"centrality\":264,\"complexity\":24.0,\"blocking factor\":15,\"delay factor\":9.0},\"nameSub\":\"Statics\",\"id\":12,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":6,\"target_id\":13,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"MATH  223\",\"metrics\":{\"centrality\":252,\"complexity\":25

In [3]:
basic_metrics(AE_degree_plan)
AE_degree_plan.metrics

Dict{String,Any} with 8 entries:
  "total credit hours"         => 129
  "avg. credits per term"      => 16.125
  "min. credits in a term"     => 15
  "term credit hour std. dev." => 0.927025
  "number of terms"            => 8
  "max. credits in a term"     => 18
  "min. credit term"           => 4
  "max. credit term"           => 3

In [4]:
CS_degree_plan = read_csv("Univ_of_Arizona-CS.csv")
visualize(CS_degree_plan, notebook=true, scale=0.8)

WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src => "http://localhost:8156",:id => "curriculum",:events => Dict{Symbol,WebIO.JSString}(:load => WebIO.JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"Computer Science\",\"dp_name\":\"2019-20 Degree Plan\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":2,\"target_id\":1,\"type\":\"CurriculumCorequisite\"}],\"name\":\"CSC 110\",\"metrics\":{\"centrality\":25,\"complexity\":12.0,\"blocking factor\":7,\"delay factor\":5.0},\"nameSub\":\"Intro to Computer Programming I\",\"id\":1,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"MATH 120R\",\"metrics\":{\"centrality\":0,\"complexity\":15.0,\"blocking factor\":10,\"delay factor\":5.0},\"nameSub\":\"Pre-Calculus\",\"id\":2,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"ENGL 101\",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"English Composition I\",\"id\":3,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"1st Semester Second Language\",\"id\":4,\"credits\":4,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":1,\"target_id\":5,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"CSC 120\",\"metrics\":{\"centrality\":25,\"complexity\":11.0,\"blocking factor\":6,\"delay factor\":5.0},\"nameSub\":\"Intro to Computer Programming II\",\"id\":5,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":2,\"target_id\":6,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"MATH 122A/B\",\"metrics\":{\"centrality\":3,\"complexity\":4.0,\"blocking factor\":1,\"delay factor\":3.0},\"nameSub\":\"Calculus I\",\"id\":6,\"credits\":5,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":3,\"target_id\":7,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"ENGL 102\",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"English Composition II\",\"id\":7,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":4,\"target_id\":8,\"type\":\"CurriculumPrerequisite\"}],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"2nd Semester Second Language\",\"id\":8,\"credits\":4,\"nameCanonical\":\"\"}],\"id\":2},{\"name\":\"Term 3\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":5,\"target_id\":9,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"CSC 210\",\"metrics\":{\"centrality\":20,\"complexity\":9.0,\"blocking factor\":4,\"delay factor\":5.0},\"nameSub\":\"Software Development\",\"id\":9,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":5,\"target_id\":10,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"CSC 245\",\"metrics\":{\"centrality\":5,\"complexity\":6.0,\"blocking factor\":1,\"delay factor\":5.0},\"nameSub\":\"Intro to Discrete Structures\",\"id\":10,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":6,\"target_id\":11,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"MATH 129\",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":0,\"delay factor\":3.0},\"nameSub\":\"Calculus II\",\"id\":11,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Tier 1 General Education 1\",\"id\":12,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay facto

In [5]:
EE_degree_plan = read_csv("Ga_Tech-EE.csv")
visualize(EE_degree_plan, notebook=true, scale=0.8)

WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src => "http://localhost:8156",:id => "curriculum",:events => Dict{Symbol,WebIO.JSString}(:load => WebIO.JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"Electrical Engineering\",\"dp_name\":\"2017-18 Plan\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":9.0,\"blocking factor\":4,\"delay factor\":5.0},\"nameSub\":\"CHEM 1310\",\"id\":1,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":23.0,\"blocking factor\":15,\"delay factor\":8.0},\"nameSub\":\"MATH 1551\",\"id\":2,\"credits\":2,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":16.0,\"blocking factor\":10,\"delay factor\":6.0},\"nameSub\":\"MATH 1553\",\"id\":3,\"credits\":2,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":17.0,\"blocking factor\":10,\"delay factor\":7.0},\"nameSub\":\"CS 1371\",\"id\":4,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Wellness Elective\",\"id\":5,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"ENGL 1101\",\"id\":6,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":2,\"target_id\":7,\"type\":\"CurriculumPrerequisite\"}],\"name\":\" \",\"metrics\":{\"centrality\":29,\"complexity\":17.0,\"blocking factor\":9,\"delay factor\":8.0},\"nameSub\":\"PHYS 2211\",\"id\":7,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":2,\"target_id\":8,\"type\":\"CurriculumPrerequisite\"}],\"name\":\" \",\"metrics\":{\"centrality\":34,\"complexity\":17.0,\"blocking factor\":10,\"delay factor\":7.0},\"nameSub\":\"MATH 1552\",\"id\":8,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":4,\"target_id\":9,\"type\":\"CurriculumPrerequisite\"}],\"name\":\" \",\"metrics\":{\"centrality\":11,\"complexity\":13.0,\"blocking factor\":6,\"delay factor\":7.0},\"nameSub\":\"ECE 2020\",\"id\":9,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"History/Government Elective\",\"id\":10,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":6,\"target_id\":11,\"type\":\"CurriculumPrerequisite\"}],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"ENGL 1102\",\"id\":11,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":2},{\"name\":\"Term 3\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":7,\"target_id\":12,\"type\":\"CurriculumPrerequisite\"}],\"name\":\" \",\"metrics\":{\"centrality\":29,\"complexity\":16.0,\"blocking factor\":8,\"delay factor\":8.0},\"nameSub\":\"PHYS 2212\",\"id\":12,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":3,\"target_id\":13,\"type\":\"CurriculumPrerequisite\"},{\"source_id\":8,\"target_id\":13,\"type\":\"CurriculumPrerequisite\"}],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":0,\"delay factor\":3.0},\"nameSub\":\"MATH 2552\",\"id\":13,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":4,\"target_id\":14,\"type\":\"CurriculumPrerequisite\"},{\"source_id\":3,\"target_id\":14,\"type\":\"CurriculumPrerequisite\"},{\"sou

In [6]:
basic_metrics(EE_degree_plan)
EE_degree_plan.metrics

Dict{String,Any} with 8 entries:
  "total credit hours"         => 134
  "avg. credits per term"      => 16.75
  "min. credits in a term"     => 15
  "term credit hour std. dev." => 1.08972
  "number of terms"            => 8
  "max. credits in a term"     => 18
  "min. credit term"           => 4
  "max. credit term"           => 5

### 2.2 Creating the Student Cohort

The following command will create an inital cohort of students `n` using a simple enrollment model. Specifically, with this simple model, all students are assumed equally likely (or unlikely) to pass a given class according the course pass rate probability model.

In [7]:
enrollment_model = Enrollment  # use the Enrollment module to determine if/when student may enroll in a course
stopouts = true  # assume that student may stop out of the cohort
n = 1000   # student cohort size will be 100
students = simple_students(n);  # create a student cohort

### 2.3 The Course Performance Model

First let's see what happens if the instructional complexity is the same for both programs. Specifically, we will use course pass rates as a proxy for instructional complexity, and we'll set the pass rates for all courses at 90%. 

In [8]:
performance_model = PassRate
real_passrate = false
set_passrates(AE_degree_plan.curriculum.courses, 0.9)
set_passrates(CS_degree_plan.curriculum.courses, 0.9)
course_passrate = 0.9;

### 2.4 Setting the Simulation Parameters

In [9]:
max_credits = 18  # the maximum number of credit hours a student may enroll in during a semester
duration_lock = false # rather than simulating until no students are left in the cohort, run for a fixed number of terms
num_terms = 12  # the maximum number of terms in the simulation
course_attempt_limit = 3;  # number of times a student may attempt a course

### 2.5 Running the Simulation 
The `simulation` function is used to execute the simulation.  Depending upon how many students are in the cohort, this may take some time to run.

In [10]:
simulation = simulate(AE_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);

In order to view the results of the simulation, use the `simulation_report` function:

In [11]:
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Aerospace Engineering, BS -- 2019-20 Degree Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 3
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 658
Graduation Rate: 65.8%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.054, 0.381, 0.604, 0.655, 0.658]
Average time to degree: 9.425531914893616 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 341
Number of Students Reaching Max Attempts: 39
Stop-out Rate: 34.1%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.073, 0.202, 0.229, 0.285, 0.306, 0.315, 0.325, 0.33, 0.336, 0.339, 0.341, 0.341]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.073, 0.202, 0.224, 0.273, 0.29, 0.295, 0.302, 0.302, 0.

Now let's run the same set of students through the Computer Science curriculum:

In [12]:
simulation = simulate(CS_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Computer Science, BS -- 2019-20 Degree Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 3
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 663
Graduation Rate: 66.3%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.272, 0.595, 0.656, 0.663, 0.0]
Average time to degree: 8.70286576168929 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 337
Number of Students Reaching Max Attempts: 29
Stop-out Rate: 33.7%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.06, 0.179, 0.215, 0.271, 0.304, 0.309, 0.322, 0.328, 0.334, 0.337, 0.337, 0.0]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.06, 0.179, 0.212, 0.267, 0.297, 0.301, 0.308, 0.308, 0.308, 0.308

In [13]:
real_passrate = false
set_passrates(EE_degree_plan.curriculum.courses, 0.9)
simulation = simulate(EE_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Electrical Engineering, BS -- 2017-18 Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 3
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 665
Graduation Rate: 66.5%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.075, 0.416, 0.62, 0.664, 0.665]
Average time to degree: 9.330827067669173 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 335
Number of Students Reaching Max Attempts: 28
Stop-out Rate: 33.5%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.076, 0.192, 0.228, 0.273, 0.298, 0.311, 0.321, 0.327, 0.331, 0.334, 0.335, 0.335]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.076, 0.192, 0.223, 0.267, 0.292, 0.301, 0.307, 0.307, 0.307, 

Next we will use a model to determine whether or not a student passes a course. We will use actual pass/fail rates computed using historical data for the courses in the degree plan shown above. 

In [14]:
course_passrate = 0.9  # use if a course is not contained in the CSV file
real_passrate = true  # use the actual pass rates, rather than course_passrate for all courses
set_passrates_from_csv(AE_degree_plan.curriculum.courses, "./Student_Grades_sp17_to_fall19.csv", course_passrate)

Note: A more realistic model for predicting student performance could be used here. Specifically, a more realistic model might:

- take student demographics into account, including the major they are in,
- take prior grades into account when predicting future grades,
- take into account factors that influence student stopout, e.g., academic standing, GPA, unment need, etc.

Learning the model pararmeters using actual student data would improve the fidelity of the simulation.

In [15]:
simulation = simulate(AE_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Aerospace Engineering, BS -- 2019-20 Degree Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 3
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 568
Graduation Rate: 56.8%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.014, 0.244, 0.487, 0.556, 0.568]
Average time to degree: 9.709507042253522 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 427
Number of Students Reaching Max Attempts: 117
Stop-out Rate: 42.7%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.09, 0.193, 0.25, 0.33, 0.364, 0.39, 0.413, 0.415, 0.42, 0.421, 0.425, 0.427]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.09, 0.193, 0.226, 0.27, 0.289, 0.3, 0.31, 0.31, 0.31, 0.31

│ 41  │ Aerospace Eng.Design                    │ 0.0%  │ 0.0%  │ 0.0%  │
│ 42  │ Num. Meth. in Fluid Mech. & Heat Xfer   │ 0.0%  │ 0.0%  │ 0.0%  │
│ 43  │ Technical Elective 1                    │ 0.0%  │ 0.0%  │ 0.0%  │
│ 44  │ Technical Elective 2                    │ 0.0%  │ 0.0%  │ 0.0%  │
│ 45  │ Tier II General Ed. 2                   │ 0.0%  │ 0.0%  │ 0.0%  │
│ 46  │ GRAD RATE                               │ 0.0%  │ 0.0%  │ 0.0%  │

│ Row │ TERM4 │ TERM5 │ TERM6 │ TERM7 │ TERM8 │ TERM9 │ TERM10 │ TERM11 │
├─────┼───────┼───────┼───────┼───────┼───────┼───────┼────────┼────────┤
│ 1   │ 98.0% │ 98.0% │ 98.0% │ 98.0% │ 98.0% │ 98.0% │ 98.0%  │ 98.0%  │
│ 2   │ 96.1% │ 96.1% │ 96.1% │ 96.1% │ 96.1% │ 96.1% │ 96.1%  │ 96.1%  │
│ 3   │ 99.1% │ 99.1% │ 99.1% │ 99.1% │ 99.1% │ 99.1% │ 99.1%  │ 99.1%  │
│ 4   │ 99.0% │ 99.0% │ 99.0% │ 99.0% │ 99.0% │ 99.0% │ 99.0%  │ 99.0%  │
│ 5   │ 97.7% │ 97.7% │ 97.7% │ 97.7% │ 97.7% │ 97.7% │ 97.7%  │ 97.7%  │
│ 6   │ 84.3% │ 84.8% │ 85.1% │ 85.1%

Now, let's see what happens if we use the actual course pass rates in the computer science program.

In [16]:
set_passrates_from_csv(CS_degree_plan.curriculum.courses, "./Student_Grades_sp17_to_fall19.csv", course_passrate)
simulation = simulate(CS_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Computer Science, BS -- 2019-20 Degree Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 3
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 545
Graduation Rate: 54.50000000000001%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.119, 0.422, 0.524, 0.545, 0.0]
Average time to degree: 9.045871559633028 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 455
Number of Students Reaching Max Attempts: 127
Stop-out Rate: 45.5%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.093, 0.214, 0.303, 0.36, 0.395, 0.419, 0.435, 0.443, 0.45, 0.454, 0.455, 0.0]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.093, 0.214, 0.251, 0.297, 0.32, 0.327, 0.329, 0.328

│ 38  │ 0.0%  │ 0.0%  │ 0.0%  │ 11.9% │ 42.2% │ 52.4%  │ 54.5%  │ 0.0%   │

## 3. What-if Analyses

These simulation capabilities allow us to conduct what-if analyses around the impact that changes to curricular structure or instructional improvements will have on student success.  First, let's consider how changing the number of allowed attempts (from 2 to 3) would impact graduation rates in both of these programs.

In [17]:
course_attempt_limit = 3
simulation = simulate(AE_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Aerospace Engineering, BS -- 2019-20 Degree Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 3
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 570
Graduation Rate: 56.99999999999999%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.018, 0.249, 0.481, 0.552, 0.57]
Average time to degree: 9.719298245614034 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 428
Number of Students Reaching Max Attempts: 111
Stop-out Rate: 42.8%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.093, 0.199, 0.258, 0.351, 0.378, 0.4, 0.414, 0.415, 0.419, 0.422, 0.427, 0.428]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.093, 0.199, 0.242, 0.295, 0.309, 0.313, 0.3

│ 45  │ 0.2%  │ 1.1%  │ 2.5%  │ 5.7%  │ 9.9%  │ 40.7% │ 54.5%  │ 57.1%  │
│ 46  │ 0.0%  │ 0.0%  │ 0.0%  │ 0.0%  │ 1.8%  │ 24.9% │ 48.1%  │ 55.2%  │

│ Row │ TERM12 │
├─────┼────────┤
│ 1   │ 96.7%  │
│ 2   │ 96.1%  │
│ 3   │ 98.6%  │
│ 4   │ 99.5%  │
│ 5   │ 97.9%  │
│ 6   │ 84.1%  │
│ 7   │ 86.7%  │
│ 8   │ 83.1%  │
│ 9   │ 87.0%  │
│ 10  │ 86.4%  │
│ 11  │ 84.9%  │
│ 12  │ 70.9%  │
│ 13  │ 71.0%  │
│ 14  │ 70.0%  │
│ 15  │ 76.6%  │
│ 16  │ 78.7%  │
│ 17  │ 76.7%  │
│ 18  │ 71.6%  │
│ 19  │ 64.1%  │
│ 20  │ 63.7%  │
│ 21  │ 63.0%  │
│ 22  │ 69.9%  │
│ 23  │ 60.7%  │
│ 24  │ 62.2%  │
│ 25  │ 60.4%  │
│ 26  │ 65.2%  │
│ 27  │ 63.2%  │
│ 28  │ 99.2%  │
│ 29  │ 59.4%  │
│ 30  │ 59.7%  │
│ 31  │ 59.5%  │
│ 32  │ 59.2%  │
│ 33  │ 99.9%  │
│ 34  │ 61.2%  │
│ 35  │ 58.7%  │
│ 36  │ 58.5%  │
│ 37  │ 58.6%  │
│ 38  │ 58.7%  │
│ 39  │ 59.1%  │
│ 40  │ 84.7%  │
│ 41  │ 58.1%  │
│ 42  │ 58.3%  │
│ 43  │ 59.6%  │
│ 44  │ 59.5%  │
│ 45  │ 57.9%  │
│ 46  │ 57.0%  │

In [18]:
simulation = simulate(CS_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Computer Science, BS -- 2019-20 Degree Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 3
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 568
Graduation Rate: 56.8%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.133, 0.451, 0.554, 0.566, 0.568]
Average time to degree: 9.0 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 432
Number of Students Reaching Max Attempts: 141
Stop-out Rate: 43.2%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.078, 0.195, 0.289, 0.337, 0.376, 0.398, 0.418, 0.425, 0.431, 0.432, 0.432, 0.432]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.078, 0.195, 0.225, 0.263, 0.284, 0.285, 0.291, 0.291, 0.291, 0.291, 0.29

Next, notice that CSC 110 -- Intro to Computer Programming I is clearly a gateway course for the computer science program at the University of Arizona, and that is has a relatively low success rate. There are a few other courses with teh CSC prefix that also have low success rates. What would happen if the instruction and instructional support were changed in a way that enabled the students taking CSC 110, CSC 120 and CSC 353 to obtain a 90% pass rate? 

In [19]:
convert_ids(CS_degree_plan.curriculum)
csc110 = course(CS_degree_plan.curriculum, "CSC", "110", "Intro to Computer Programming I", "")
csc120 = course(CS_degree_plan.curriculum, "CSC", "120", "Intro to Computer Programming II", "")
csc210 = course(CS_degree_plan.curriculum, "CSC", "210", "Software Development", "")
csc345 = course(CS_degree_plan.curriculum, "CSC", "345", "Analysis of Discrete Structures", "")
csc352 = course(CS_degree_plan.curriculum, "CSC", "352", "Systems Programming & Unix", "")

csc110.passrate = csc120.passrate = csc210.passrate = csc345.passrate = csc352.passrate = 0.9;

In [20]:
simulation = simulate(CS_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Computer Science, BS -- 2019-20 Degree Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 3
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 611
Graduation Rate: 61.1%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.186, 0.51, 0.599, 0.611, 0.0]
Average time to degree: 8.880523731587562 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 389
Number of Students Reaching Max Attempts: 71
Stop-out Rate: 38.9%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.079, 0.197, 0.255, 0.31, 0.345, 0.364, 0.377, 0.384, 0.385, 0.388, 0.389, 0.0]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.079, 0.197, 0.234, 0.281, 0.304, 0.313, 0.318, 0.318, 0.318, 0.31

├─────┼───────┼───────┼───────┼───────┼───────┼────────┼────────┼────────┤
│ 1   │ 99.2% │ 99.2% │ 99.2% │ 99.2% │ 99.2% │ 99.2%  │ 99.2%  │ 99.2%  │
│ 2   │ 94.8% │ 94.8% │ 94.8% │ 94.8% │ 94.8% │ 94.8%  │ 94.8%  │ 94.8%  │
│ 3   │ 99.1% │ 99.1% │ 99.1% │ 99.1% │ 99.1% │ 99.1%  │ 99.1%  │ 99.1%  │
│ 4   │ 90.6% │ 90.6% │ 90.6% │ 90.6% │ 90.6% │ 90.6%  │ 90.6%  │ 90.6%  │
│ 5   │ 89.6% │ 89.6% │ 89.6% │ 89.6% │ 89.6% │ 89.6%  │ 89.6%  │ 89.6%  │
│ 6   │ 84.9% │ 84.9% │ 84.9% │ 84.9% │ 84.9% │ 84.9%  │ 84.9%  │ 84.9%  │
│ 7   │ 87.8% │ 87.9% │ 87.9% │ 87.9% │ 87.9% │ 87.9%  │ 87.9%  │ 87.9%  │
│ 8   │ 78.9% │ 78.9% │ 78.9% │ 78.9% │ 78.9% │ 78.9%  │ 78.9%  │ 78.9%  │
│ 9   │ 78.2% │ 78.3% │ 78.3% │ 78.3% │ 78.3% │ 78.3%  │ 78.3%  │ 78.3%  │
│ 10  │ 75.2% │ 76.3% │ 76.4% │ 76.4% │ 76.4% │ 76.4%  │ 76.4%  │ 76.4%  │
│ 11  │ 71.0% │ 73.1% │ 73.5% │ 73.6% │ 73.6% │ 73.6%  │ 73.6%  │ 73.6%  │
│ 12  │ 98.6% │ 98.6% │ 98.6% │ 98.6% │ 98.6% │ 98.6%  │ 98.6%  │ 98.6%  │
│ 13  │ 77.4% │ 77.8% │ 7

# References

Heileman, G. L., Abdallah, C.T., Slim, A., and Hickman, M. (2018). Curricular analytics: A framework for quantifying the impact of curricular reforms and pedagogical innovations. www.arXiv.org, arXiv:1811.09676 [cs.CY].

Hickman, M. (2014). Development of a Curriculum Analysis and Simulation Library with Applications in Curricular Analytics. MS thesis, University of New Mexico,
Albuquerque, NM.